In [10]:
import numpy as np
from sklearn.mixture import BayesianGaussianMixture
from scipy.spatial.distance import cdist
from sklearn.decomposition import PCA

def gaussian_mixture_k_segs(data, n_components=5):
    """
    Approximate a principal curve using a Gaussian Mixture Model.
    
    Parameters:
    - data: np.ndarray, shape (n_samples, n_features)
      Input data.
    - n_components: int
      Number of Gaussian components in the GMM.
    - n_points: int
      Number of points to sample on the principal curve.

    Returns:
    - curve: np.ndarray, shape (n_points, n_features)
      The computed principal curve.
    """
    gmm = BayesianGaussianMixture(
        n_components=n_components,
        max_iter=100,
        covariance_type="full",
        weight_concentration_prior_type="dirichlet_process",
        tol=1e-4,
        random_state=2,
        verbose=2,       
        init_params="k-means++",
        n_init=2,
        warm_start=True
    )
    gmm.fit(data)
    means = gmm.means_
    covariances = gmm.covariances_
    return means, covariances

def create_segs_from_model(means, covariances):
    """
    Cria os segmentos e as direções de segunda maior variação a partir das médias e covariâncias
    """
    pcs_ortogonal = []
    segments = []
    for i, mean in enumerate(means):
        # Compute PCA on the covariance matrix
        eigenvalues, eigenvectors = np.linalg.eigh(covariances[i])
        
        # Ordenar os índices dos valores em ordem decrescente
        sorted_vals_idx = np.argsort(eigenvalues)[::-1]
        sorted_vecs_idx = np.argsort(eigenvectors)[::-1]
        
        '''
        O multiplicador de confiança é um metaparâmetro que multiplica o tamanho de cada segmento.
        É equivalente a multiplicar o desvio padrão local por 1, 2, etc para se elevar a confiança
        Daria para determinar este número  iterativamente, fazendo-se uma média do comprimento dos segmentos versus ligações e buscando minimizar a diferença 
        '''
        confidence_multiplier = 2
        
        # Pegue os dois maiores autovalores
        val_index1, val_index2 = sorted_vals_idx[:2]
        pc_1_norm, pc_2_norm = confidence_multiplier*np.sqrt(eigenvalues[val_index1]), confidence_multiplier*np.sqrt(eigenvalues[val_index2])
        
        vec_index1, vec_index2 = sorted_vecs_idx[:2]
        pc_1_dir, pc_2_dir = eigenvalues[vec_index1], eigenvalues[vec_index2]
                
        segment = np.array([mean + pc_1_dir * pc_1_norm, mean - pc_1_dir * pc_1_norm])  
        ortogonal = np.array([mean + pc_2_dir * pc_2_norm, mean - pc_2_dir * pc_2_norm])  
  
        # Save ellipse data
        segments.append(segment)
        pcs_ortogonal.append(ortogonal)
    return segments, pcs_ortogonal

def order_segments(segments, ortogonal_components):
    '''
    Ordena Segmentos e também a componente principal ortogonal  
    '''
    # Lista para armazenar a ordem dos segmentos
    ordered_segments = []

    # Usar o primeiro segmento como ponto inicial
    current_segment = segments[0]
    ordered_segments.append(current_segment)
    ordered_pcs = [ortogonal_components[0]]
    current_start, current_end = current_segment

    # Criar uma lista dos segmentos restantes
    remaining_segments = list(segments[1:])
    remaining_pcs = list(ortogonal_components[1:])


    while remaining_segments:
        # Criar uma lista com todos os extremos dos segmentos restantes
        candidates = []
        for seg in remaining_segments:
            candidates.extend(seg)
        candidates = np.array(candidates)
        
        # Calcular a menor distância entre o ponto final do segmento atual e os candidatos
        distances_end = cdist([current_end], candidates)
        distances_start = cdist([current_start], candidates)
        min_idx_end = np.argmin(distances_end)
        min_dist_end = np.min(distances_end)
        min_idx_start = np.argmin(distances_start)
        min_dist_start = np.min(distances_start)
        
        if min_dist_start > min_dist_end:  
            next_segment_idx = min_idx_end // 2
            next_segment = remaining_segments[next_segment_idx]
            ordered_pcs.append(remaining_pcs[next_segment_idx])
            if min_idx_end % 2 == 0:
                current_end = next_segment[1] 
                ordered_segments.append(next_segment)
            else:
                current_end = next_segment[0] 
                ordered_segments.append([next_segment[1], next_segment[0]])
                
        else:
            next_segment_idx = min_idx_start // 2
            next_segment = remaining_segments[next_segment_idx]
            ordered_pcs.insert(0, remaining_pcs[next_segment_idx])
            if min_idx_start % 2 == 0:
                current_start = next_segment[1] 
                ordered_segments.insert(0, [next_segment[1], next_segment[0]])
            else:
                current_start = next_segment[0] 
                ordered_segments.insert(0, next_segment)
            
        # Remover o segmento já utilizado
        del remaining_segments[next_segment_idx]
        del remaining_pcs[next_segment_idx]

    return np.array(ordered_segments), ordered_pcs

def calc_connection_seg(seg_prev, pc_prev, seg_current, pc_current):
    pc_norm = (np.linalg.norm(pc_current) + np.linalg.norm(pc_prev)) / 2
    return {
        'max_dist': pc_norm,
        'seg_points': np.array([seg_prev[1], seg_current[0]]),
        'is_conn': True
    }   
    
def extract_final_curve(ordered_segments, ordered_pcs):
    final_curve = []
    for i, segment in enumerate(ordered_segments):                
        final_curve.append({
            'ortogonal_pc': ordered_pcs[i],
            'seg_points': segment,
            'is_conn': False
        }) 
        if (i>=1 and i < len(ordered_segments)):
            conn_seg = calc_connection_seg(ordered_segments[i-1], ordered_pcs[i-1], segment, ordered_pcs[i])        
            final_curve.append(conn_seg)
    return final_curve
        
'''
Aqui, uma ideia interessante seria usarmos a distância de Mahalanobis,
utilizando os dados estatísticos estimados pelo gausian mixture para cada segmento.
Talvez dê uma medida mais exada da proximidade de um dado a uma curva/segmento e melhore a performance
'''    
def calc_min_distances(points, segments):
    """
    Calculate the minimum distance from each point to multiple segments and store the segment index.

    Parameters:
    - points: Array of shape (n, d), where each row is a point in N-dimensional space.
    - segments: Array of shape (m, 2, d), where each segment is defined by two points.

    Returns:
    - min_distances: Array of shape (n,), where each entry is the minimum distance for a point.
    - segment_indices: Array of shape (n,), where each entry is the index of the closest segment.
    """
    points = np.array(points)  # Shape: (n, d)
    segments = np.array(segments)  # Shape: (m, 2, d)

    a = segments[:, 0, :]  # Start points of segments, shape: (m, d)
    b = segments[:, 1, :]  # End points of segments, shape: (m, d)

    # Vector from A to B (segment direction vectors), shape: (m, d)
    ab = b - a

    # Squared length of each segment, shape: (m,)
    ab_len_sq = np.sum(ab**2, axis=1)

    # Expand points to shape (n, m, d)
    p_exp = points[:, np.newaxis, :]  # Shape: (n, 1, d)
    a_exp = a[np.newaxis, :, :]  # Shape: (1, m, d)
    ab_exp = ab[np.newaxis, :, :]  # Shape: (1, m, d)

    # Vector from A to P, shape: (n, m, d)
    ap = p_exp - a_exp

    # Projection factors (t values), shape: (n, m)
    t = np.sum(ap * ab_exp, axis=2) / ab_len_sq

    # Clamp t to the range [0, 1]
    t = np.clip(t, 0, 1)

    # Closest points on the segments, shape: (n, m, d)
    closest_points = a_exp + t[:, :, np.newaxis] * ab_exp

    # Distances from points to the closest points on each segment, shape: (n, m)
    distances = np.linalg.norm(p_exp - closest_points, axis=2)

    # Minimum distances and corresponding segment indices
    min_distances = np.min(distances, axis=1)

    return min_distances

def gaussian_mixture_principal_curve(data, k):
    final_curve = []
    means, covariances = gaussian_mixture_k_segs(data, k)
    segments, pcs_ortogonal = create_segs_from_model(means, covariances)
    ordered_segments, ordered_pcs = order_segments(segments, pcs_ortogonal)
    final_curve = extract_final_curve(ordered_segments, ordered_pcs)
    curve_segments = np.array([seg['seg_points'] for seg in final_curve])
    return curve_segments, final_curve  

import pandas as pd
file_path = '../data/digit-recognizer/train.csv' 
digits_images = pd.read_csv(file_path)

curves = [];
pcas = []
for digit in range(10):
    print(f"Criando a curva para o digito: {digit}")
    class_data = digits_images[digits_images["label"] == digit]
    pca = PCA(n_components=0.98, random_state=2)
    data_reduced = pca.fit_transform(class_data.to_numpy()[:, 1:])
    pcas.append(pca)
    pca2 = PCA().fit(data_reduced)
    cumulative_variance = np.cumsum(pca2.explained_variance_ratio_)
    n_components_95 = (np.argmax(cumulative_variance >= 0.98) + 1)
    print("Número de segmentos: ", n_components_95)
    curve_segments, final_curve = gaussian_mixture_principal_curve(data_reduced, n_components_95)
    curves.append((curve_segments, final_curve))

Criando a curva para o digito: 0
Número de segmentos:  136
Initialization 0
Initialization converged. time lapse 12.93256s	 lower bound -20324906.81863.
Initialization 1
Initialization converged. time lapse 13.04088s	 lower bound -20324140.14444.
Criando a curva para o digito: 1
Número de segmentos:  83
Initialization 0
Initialization converged. time lapse 4.15442s	 lower bound -6351977.57800.
Initialization 1
Initialization converged. time lapse 8.03858s	 lower bound -6343669.76379.
Criando a curva para o digito: 2
Número de segmentos:  164
Initialization 0
Initialization converged. time lapse 15.07085s	 lower bound -30740559.40487.
Initialization 1
Initialization converged. time lapse 15.12295s	 lower bound -30737653.02365.
Criando a curva para o digito: 3
Número de segmentos:  158
Initialization 0
Initialization converged. time lapse 15.58051s	 lower bound -27486352.10715.
Initialization 1
Initialization converged. time lapse 18.99287s	 lower bound -27487487.85196.
Criando a curva p

In [11]:
''' Validação '''
num_samples = 1000
validation = digits_images.head(num_samples).copy()
validation_np = validation.to_numpy()[:, 1:]
from tqdm import tqdm

for curve_idx in tqdm(range(len(curves)),desc="Calculando menor distância", ncols=80, colour="green"): 
    segments, metadata = curves[curve_idx] # ainda não usamos a segunda componente principal
    pca = pcas[curve_idx]
    min_dists = calc_min_distances(pca.transform(validation_np), segments)
    column_name = curve_idx
    validation[column_name] = min_dists

columns_of_interest = [idx for idx in range(10)]
validation.loc[:, "guess"] = validation[columns_of_interest].idxmin(axis=1)
sum_correct_all_columns = validation.loc[validation["label"] == validation["guess"]]
print(sum_correct_all_columns.shape[0] / num_samples * 100)

Calculando menor distância: 100%|███████████████| 10/10 [00:12<00:00,  1.21s/it]

93.8


In [12]:
''' Gerar submissão '''
from tqdm import tqdm

file_path = '../data/digit-recognizer/test.csv' 
test_data = pd.read_csv(file_path)
data_points = test_data.to_numpy()
for curve_idx in tqdm(range(len(curves)),desc="Calculando menor distância", ncols=80, colour="green"): 
    segments, metadata = curves[curve_idx] # ainda não usamos a segunda componente principal
    pca = pcas[curve_idx]
    min_dists = calc_min_distances(pca.transform(data_points), segments)
    column_name = curve_idx
    test_data[column_name] = min_dists

columns_of_interest = [idx for idx in range(10)]
test_data.loc[:, "Label"] = test_data[columns_of_interest].idxmin(axis=1)
answer = test_data[["Label"]].copy()  
answer["ImageId"] = range(1, len(answer) + 1)  
answer = answer[["ImageId", "Label"]]

# Save to CSV
output_file = "submission.csv"
answer.to_csv(output_file, index=False)

Calculando menor distância:   0%|                        | 0/10 [01:37<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
'''
Code to generate synthetic data
'''
# def generate_data_gaussians():    
#     # Configurações das gaussianas
#     np.random.seed(42)  # Para reprodutibilidade
#     num_gaussians = 5
#     points_per_gaussian = 1000

#     # Parâmetros de cada gaussiana (média e covariância)
#     means = [
#         [0, 0],
#         [5, 3],
#         [10, 2],
#         [15, 1],
#     ]

#     covariances = [
#         [[2, 3], [0, 0.5]],
#         [[5,0], [0.2, 0.5]],
#         [[3, -2], [0, 0]],
#         [[1.2, 0.6], [0.6, 1]],
#     ]

#     # Gerar os dados
#     data = []
#     for mean, cov in zip(means, covariances):
#         gaussian_points = np.random.multivariate_normal(mean, cov, points_per_gaussian)
#         data.append(gaussian_points)

#     data = np.vstack(data)

#     # Adicionar ruído
#     noise = np.random.normal(0, 0.2, data.shape)
#     data_with_noise = data + noise
#     return data_with_noise

# def load_spiral_data():
#     import pandas as pd 
#     file_path = '../data/spiral_data.csv'  # Update the path if necessary
#     spiral_data = pd.read_csv(file_path)

#     # Extract x and y coordinates
#     x_data = spiral_data['x']
#     y_data = spiral_data['y']
#     data_with_noise = np.column_stack((x_data, y_data))
#     return data_with_noise